In [60]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import geopandas as gpd
import time
import random
import numpy as np
import re
from gmplot import gmplot
import folium
from fake_useragent import FakeUserAgent
from coordTransform_utils import bd09_to_wgs84 as coord_trans

# 爬虫

In [7]:
#数据结构
"""
-title
-price
-info
  -小区
  -面积
  -朝向
  -房间
-label
"""

In [61]:
#爬虫程序

# url_2='https://hz.lianjia.com/ditiezufang/li189500791680395/pg{}rt200600000001/#contentList'.format(page) #2号线
# url_5='https://hz.lianjia.com/ditiezufang/li1820044952055481/pg{}rt200600000001/#contentList'.format(page) #5号线
urls=['https://hz.lianjia.com/ditiezufang/li189500791680395/pg{}rt200600000001/#contentList'.format(page) for page in range(1,101)]\
+['https://hz.lianjia.com/ditiezufang/li1820044952055481/pg{}rt200600000001/#contentList'.format(page) for page in range(1,101)]


def main(url):
    html=requests.get(url).text
    soup=bs(html,'lxml')
    
    hrefs=['https://hz.lianjia.com'+x for x in [y.a['href'] for y in soup.find_all('p',class_="content__list--item--title twoline")]]
    titles=[x.text.strip() for x in soup.find_all('p',class_="content__list--item--title twoline")]
    prices=[x.em.text for x in soup.find_all('span',class_="content__list--item-price")]
    labels=[x.text.strip().replace('\n',' ') for x in soup.find_all('p',class_="content__list--item--bottom oneline")]
    infos=[x.text.strip().replace('\n','').replace(' ','') for x in soup.find_all('p',class_="content__list--item--des")]
    
    
    #解析信息
    def pick(index,info):
        try:
            return info.split('/')[index]
        except:
            return 
        
    locs=[pick(0,x) for x in infos]
    squares=[pick(1,x) for x in infos]
    directions=[pick(2,x) for x in infos]
    rooms=[pick(3,x) for x in infos]
    
    
    
    
    df=pd.DataFrame({'title':titles,
                    'price':prices,
                    'loc':locs,
                     'square':squares,
                     'direction':directions,
                     'room':rooms,
                    'label':labels,
                     'link':hrefs})
    
    df.to_csv(r'lianjia_rent.csv',mode='a',encoding='gbk',index=None)
    

for url in urls:
    main(url)
    time.sleep(random.randint(0,5)*0.02)

# 数据加工

In [37]:
df=pd.read_excel(r'lianjia_rent_coord.xlsx')

In [16]:
#加上经纬度
# ua=FakeUserAgent()


# ips = open('proxies.txt','r').read().split('\n')

# def get_random_ip():
#     ip = random.choice(ips)
#     pxs = {ip.split(':')[0]:ip}
#     return pxs

def get_loc(url):
    try:
        html=requests.get(url).text
        longitude=re.findall(r'longitude..............*?',html)[0].replace('longitude: \'','')
        latitude=re.findall(r'latitude............*?',html)[0].replace('latitude: \'','')
        return longitude,latitude
    except:
        return 
    
    
def function(row):
    if ',' in str(row['coord']):
        return row['coord']
    else:
        return get_loc(row['link'])
    
df['coord']=df.apply(function,axis=1)
df.to_excel(r'tem.xlsx',index=None)

In [39]:
#坐标转换  bd--->84
df['coord_84']=df['coord'].apply(lambda x:coord_trans(float(x.split(',')[0]),float(x.split(',')[1])))
df.to_excel(r'tem.xlsx',index=None)

In [27]:
# 加工地铁站信息
df=pd.read_excel(r'metro.xlsx')

df['lat_84']=df['coord_bd'].apply(lambda x:coord_trans(float(x.split(',')[0]),float(x.split(',')[1]))[1])
df['lon_84']=df['coord_bd'].apply(lambda x:coord_trans(float(x.split(',')[0]),float(x.split(',')[1]))[0])

df.to_excel(r'metro.xlsx',index=None)

# 数据可视化

In [ ]:
"""
房价需要分等级：
0-1500
1500-2000
2000-2500
2500-3000
3000-3500
3500-4000
4000以上
非数字
"""

In [56]:
#读取基础数据
df=pd.read_excel(r'lianjia_rent_coord.xlsx')

In [6]:
df.head()

,title,price,loc,square,direction,room,label,link,coord,coord84_x,coord84_y
0,整租·金家渡中苑 1室0厅 南,880,余杭-三墩-金家渡中苑,10㎡,南,1室0厅1卫,公寓 月租 近地铁 押一付一 随时看房,https://hz.lianjia.com/zufang/HZ24863665196808...,"120.099058,30.346064",120.087766,30.342283
1,整租·万科劝学里 1室1厅 西南,1200,余杭-良渚-万科劝学里,27㎡,西南,1室1厅1卫,近地铁,https://hz.lianjia.com/zufang/HZ24073504213256...,"120.019819,30.354807",120.008675,30.350941
2,独栋·驻客公寓 杭州西溪银泰城店 特价房 开间,1200,仅剩1间,15㎡,1间在租,1室0厅1卫,独栋公寓 近地铁 押一付一,https://hz.lianjia.com/apartment/2672.html,"120.077927,30.293594",120.066523,30.290173
3,整租·万科劝学里 1室1厅 南,1300,余杭-良渚-万科劝学里,27㎡,南,1室1厅1卫,近地铁,https://hz.lianjia.com/zufang/HZ24164819564951...,"120.019819,30.354807",120.008675,30.350941
4,整租·西溪北苑北区 1室0厅 北,1300,余杭-未来科技城-西溪北苑北区,20㎡,北,1室0厅1卫,公寓 近地铁 精装 押一付一 随时看房,https://hz.lianjia.com/zufang/HZ24998189141047...,"120.044339,30.298439",120.033020,30.294928


In [94]:
#底图
map=folium.Map(location=[30.258605, 120.157405])

In [95]:
#把地铁站点打出来
df_metro=pd.read_excel(r'metro.xlsx')


for lat,lon,label,line in zip(df_metro['lat_84'],df_metro['lon_84'],df_metro['name'],df_metro['line']):
    if line==2:
        color='#ffc71e'
    else:
        color='#2ce7cf'
    folium.Circle(location=[lat,lon],tooltip='500m circle',radius=500,color=color).add_to(map)
    folium.Circle(location=[lat,lon],tooltip='300m circle',radius=300,color=color).add_to(map)
    folium.Marker(location=[lat,lon],tooltip=label,color=color).add_to(map)

map

In [96]:
thresholds=[0,1500,2000,2500,3000,3500,4000]
thresholds=[(thresholds[i],thresholds[i+1]) for i in range(0,6)]
colors=['#b15928','#6a3d9a','#1f78b4','#33a02c','#ffff99','#ff7f00','red','#666666']

latitudes = df.coord84_y
longitudes = df.coord84_x
labels = ['title:{}\nprice:{}元\nsquare:{}\ndirection:{}\nroom:{}'.format(a,b,c,d,e) for a,b,c,d,e 
          in zip(df.title,df.price,df.square,df.direction,df.room)]
prices= df.price

for lat, lng, label, price in zip(latitudes, longitudes, labels,prices):
    try:
        price=int(price)
        for i in range(0,6):
            if thresholds[i][0]<price<=thresholds[i][1]:
                color=colors[i]
                break
            elif i==5:
                color=colors[-2]
    except:
        color=colors[-1]
    folium.CircleMarker([lat, lng], 
                        tooltip=label,
                        radius=6, # define how big you want the circle markers to be
                        color=color,
                        fill=True,
                        fill_color=color,
                        fill_opacity=0.75,
                        line_opacity=1).add_to(map)    

map.save(r'house_rent_distributions.html')

In [16]:
# 空间可视化 demo code 
# Place map
gmap = gmplot.GoogleMapPlotter(30.354807,120.019819, 13)

# Polygon
# golden_gate_park_lats, golden_gate_park_lons = zip(*[
#     (37.771269, -122.511015),
#     (37.773495, -122.464830),
#     (37.774797, -122.454538),
#     (37.771988, -122.454018),
#     (37.773646, -122.440979),
#     (37.772742, -122.440797),
#     (37.771096, -122.453889),
#     (37.768669, -122.453518),
#     (37.766227, -122.460213),
#     (37.764028, -122.510347),
#     (37.771269, -122.511015)
#     ])
# gmap.plot(golden_gate_park_lats, golden_gate_park_lons, 'cornflowerblue', edge_width=10)

# Scatter points
lons, lats = zip(*[
    (120.099058,30.346064),
    (120.019819,30.354807),
    (120.077927,30.293594),
    (120.019819,30.354807)
    ])
gmap.scatter(lats, lons, 'red', size=40, marker=False)

# # Marker
# hidden_gem_lat, hidden_gem_lon = 37.770776, -122.461689
# gmap.marker(hidden_gem_lat, hidden_gem_lon, 'cornflowerblue')

# Draw
gmap.draw("my_map.html")

In [71]:
gdf=gpd.GeoDataFrame.from_file(r'D:\杭州生活小技巧\lianjia爬虫\data\lianjia_rent.shp',encoding='gbk')
gdf

,title,price,loc,square,direction,room,label,link,coord,coord84_x,coord84_y,geometry
0,整租·金家渡中苑 1室0厅 南,880.0,余杭-三墩-金家渡中苑,10㎡,南,1室0厅1卫,公寓 月租 近地铁 押一付一 随时看房,https://hz.lianjia.com/zufang/HZ24863665196808...,"120.099058,30.346064",120.087766,30.342283,POINT (120.087766212034 30.3422827358905)
1,整租·万科劝学里 1室1厅 西南,1200.0,余杭-良渚-万科劝学里,27㎡,西南,1室1厅1卫,近地铁,https://hz.lianjia.com/zufang/HZ24073504213256...,"120.019819,30.354807",120.008675,30.350941,POINT (120.008675445755 30.350940524161)
2,独栋·驻客公寓 杭州西溪银泰城店 特价房 开间,1200.0,仅剩1间,15㎡,1间在租,1室0厅1卫,独栋公寓 近地铁 押一付一,https://hz.lianjia.com/apartment/2672.html,"120.077927,30.293594",120.066523,30.290173,POINT (120.066523112552 30.2901734260104)
3,整租·万科劝学里 1室1厅 南,1300.0,余杭-良渚-万科劝学里,27㎡,南,1室1厅1卫,近地铁,https://hz.lianjia.com/zufang/HZ24164819564951...,"120.019819,30.354807",120.008675,30.350941,POINT (120.008675445755 30.350940524161)
4,整租·西溪北苑北区 1室0厅 北,1300.0,余杭-未来科技城-西溪北苑北区,20㎡,北,1室0厅1卫,公寓 近地铁 精装 押一付一 随时看房,https://hz.lianjia.com/zufang/HZ24998189141047...,"120.044339,30.298439",120.033020,30.294928,POINT (120.033020443434 30.2949279361239)
5,整租·金城花园 1室1厅 南,1350.0,萧山-萧山市区-金城花园,35㎡,南,1室1厅1卫,近地铁,https://hz.lianjia.com/zufang/HZ24758655829782...,"120.282082,30.163044",120.271076,30.159591,POINT (120.271076210126 30.1595910169051)
6,整租·众安花园 1室0厅 东南,1360.0,萧山-萧山市区-众安花园,20㎡,东南,1室0厅1卫,公寓 月租 近地铁 精装 押一付一 随时看房,https://hz.lianjia.com/zufang/HZ23811227978010...,"120.282564,30.170185",120.271559,30.166741,POINT (120.271559066762 30.1667409661813)
7,整租·众安花园 1室1厅 南,1360.0,萧山-萧山市区-众安花园,25㎡,南,1室1厅1卫,公寓 月租 近地铁 精装 押一付一 随时看房,https://hz.lianjia.com/zufang/HZ23731582149182...,"120.283522,30.169615",120.272512,30.166187,POINT (120.272512071106 30.166187006056)
8,整租·铭雅苑东区 1室0厅 南,1400.0,余杭-三墩-铭雅苑东区,28㎡,南,1室0厅1卫,公寓 月租 近地铁 精装 押一付一 随时看房,https://hz.lianjia.com/zufang/HZ24863811208253...,"120.092745,30.345274",120.081421,30.341614,POINT (120.081421046184 30.3416139209449)
9,独栋·一清小筑公寓 湖墅南路店 朝东单人间 开间,1400.0,仅剩1间,12㎡,1间在租,1室0厅1卫,独栋公寓 月租 近地铁 押一付一,https://hz.lianjia.com/apartment/20999.html,"120.154905,30.295378",120.143688,30.291661,POINT (120.143687634999 30.2916610146788)


In [72]:
bins = list(gdf['price'].quantile([0, 0.25, 0.5, 0.75, 1]))

m = folium.Map(location=[30.258605, 120.157405], zoom_start=12)

folium.Choropleth(
    geo_data=gdf,
#     data=gdf
#     columns=['State', 'Unemployment'],
#     key_on='feature.id',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.5,
#     legend_name='Unemployment Rate (%)',
#     bins=bins,
#     reset=True
).add_to(m)

m

RuntimeError: b'no arguments in initialization list'